In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pli
import seaborn as sns
import os
import tensorflow as tf
from PIL import Image
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import classification_report

In [ ]:
def clean_colab_memory(path):
    !rm -r {path}

def save_to_drive(src, dest):
    !cp -r {src} {dest}

# Esempio di utilizzo
# clean_colab_memory('/content/nih_chest_xray_subset')
# save_to_drive('/content/nih_chest_xray_subset', '/content/drive/My Drive/')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

labels_path = "/content/drive/MyDrive/nih_chest_xray_subset/sample_labels.csv"
labels_df = pd.read_csv(labels_path)

In [ ]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


In [ ]:
labels_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [ ]:
labels_df.shape

(5606, 11)

In [ ]:
labels_df['Finding Labels'].value_counts()

,count
Finding Labels,
No Finding,3044
Infiltration,503
Effusion,203
Atelectasis,192
Nodule,144
...,...
Atelectasis|Edema|Effusion|Infiltration|Pneumonia,1
Atelectasis|Consolidation|Edema|Infiltration|Pneumonia,1
Atelectasis|Effusion|Hernia,1


In [ ]:
data = []

for _, row in labels_df.iterrows():
    image_path = os.path.join('/content/drive/MyDrive/nih_chest_xray_subset/images', row['Image Index'])
    labels = row['Finding Labels'].split('|')
    data.append({'image_path': image_path, 'labels': labels})

print(data[:5])

[{'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_005.png', 'labels': ['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_026.png', 'labels': ['Cardiomegaly', 'Emphysema']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000017_001.png', 'labels': ['No Finding']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000030_001.png', 'labels': ['Atelectasis']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000032_001.png', 'labels': ['Cardiomegaly', 'Edema', 'Effusion']}]


In [ ]:
num_class_1 = sum(["Infiltration" in r["labels"] for r in data])
print(num_class_1)

967


In [ ]:
num_class_0 = sum(["Infiltration" not in r["labels"] for r in data])
print(num_class_0)

4639


In [ ]:
class_1_data = [r for r in data if "Infiltration" in r["labels"]]

print(f"Esempi della classe 1 disponibili per augmentation: {len(class_1_data)}")

Esempi della classe 1 disponibili per augmentation: 967


In [ ]:
def preprocess_item(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, label

paths = [item['image_path'] for item in data]
labels = [1 if "Infiltration" in item["labels"] else 0 for item in data]

# Preprocessing con parallelismo
dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
dataset = dataset.map(lambda x, y: preprocess_item(x, y), num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.cache()
dataset_nobatch = dataset
# Shuffle per migliorare il training
dataset = dataset.shuffle(buffer_size=1000)  # Cambia il buffer in base al dataset

# Creazione dei batch # Prefetch per caricare il batch successivo in parallelo
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
for images, labels in dataset.take(1):
    print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")

Batch shape: (64, 224, 224, 3), Labels shape: (64,)


In [ ]:
# loaded_dataset = tf.data.Dataset.load("/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

In [ ]:
#total_images = 0
#for images, labels in dataset:
#   total_images += images.shape[0]
#
#print(f"Total_number_of: {total_images}")

In [ ]:
# tf.data.experimental.save(dataset,"/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

# TensorFlow Keras

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False # don't update base_model weights during training

model = tf.keras.Sequential([base_model, tf.keras.layers.GlobalAveragePooling2D(), tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
dataset_size = len(dataset)

test_size = int(dataset_size * 0.2)
validation_size = int(dataset_size * 0.1)
train_size = dataset_size - test_size - validation_size

test_set = dataset.take(test_size)  # Primi 20% per il test
validation_set = dataset.skip(test_size).take(validation_size)  # Successivi 20% per validazione
train_set = dataset.skip(test_size + validation_size)  # Resto per il training


In [ ]:
print(f"Dataset totale (batch): {dataset_size}")
print(f"Test set (batch): {test_size}")
print(f"Validation set (batch): {validation_size}")
print(f"Training set (batch): {train_size}")

Dataset totale (batch): 88
Test set (batch): 17
Validation set (batch): 8
Training set (batch): 63


In [ ]:
history = model.fit(train_set, epochs=5, validation_data = validation_set)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 309s 4s/step - accuracy: 0.8224 - loss: 0.4541 - val_accuracy: 0.8652 - val_loss: 0.3916
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 207s 3s/step - accuracy: 0.8204 - loss: 0.4506 - val_accuracy: 0.8633 - val_loss: 0.3935
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 255s 3s/step - accuracy: 0.8315 - loss: 0.4330 - val_accuracy: 0.8672 - val_loss: 0.3661
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.8296 - loss: 0.4249 - val_accuracy: 0.8496 - val_loss: 0.4157
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 243s 4s/step - accuracy: 0.8215 - loss: 0.4439 - val_accuracy: 0.8477 - val_loss: 0.4217


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.8676 - loss: 0.3951
Test Loss: 0.4276522397994995, Test Accuracy: 0.8428308963775635


In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

print(classification_report(y_true, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
              precision    recall  f1-score   support

           0       0.86      0.99      0.92       932
           1       0.53      0.05      0.09       156

    accuracy                           0.86      1088
   macro avg       0.70      0.52      0.51      1088
weighted avg       0.81      0.86      0.80      1088



This is due to the imbalance of the dataset:  
the model almost always predicts 0, largely ignoring class 1.  
On class 1 we experienced:  
Precision = 50%: The model makes few predictions like 1, but half of these predictions are correct.  
Recall = 2%: The model identifies only 2% of the real class 1 examples

# Pytorch

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CovNet(nn.Module):
  def __init__(self, channels, classes):
    super(CovNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=channels, out_channels=32, kernel_size=(5,5), padding='same')
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5), padding='same')
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(in_features=64*56*56, out_features=128)
    self.relu3 = nn.ReLU()

    self.fc2 = nn.Linear(in_features=128, out_features=classes)
    self.softmax = nn.Softmax(dim=1)

  def forward_pass(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = self.maxpool1(x)
    x = nn.functional.relu(self.conv2(x))
    x = self.maxpool2
    x = self.flatten(x)
    x = nn.functional.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.softmax(x)
    return x

In [ ]:
num_classes = 2
num_epochs = 10
batch_size = 32
learning_rate = 0.001

In [ ]:
model = CovNet(input,channels=3, classes=num_classes).to(device)

In [ ]:
train_dataset = #Dataset
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = #Dataset
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
history = {'train_loss': [],
    'train_acc': [],
    'test_loss': [],
    'test_acc': []}

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_size, (data, targets) in enumerate(tqdm(train_dataset)):

        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = loss(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

# DATA AUGMENTATION

In [ ]:
# Filtra solo immagini di classe 1
class_1_data = [item for item in data if "Infiltration" in item["labels"]]
class_1_paths = [item['image_path'] for item in class_1_data]
class_1_labels = [1] * len(class_1_data)

class_1_dataset = tf.data.Dataset.from_tensor_slices((class_1_paths, class_1_labels))

def augment_image(image):
    # Operazioni di augmentazione
    image = tf.image.random_flip_left_right(image)  # Flip orizzontale
    image = tf.image.random_flip_up_down(image)    # Flip verticale
    image = tf.image.rot90(image, k=np.random.randint(1, 4))  # Rotazione casuale
    return image

# Applica augmentation agli esempi di classe 1
def preprocess_and_augment(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    image = augment_image(image)  # Applica augmentation
    return image, label

# Conta gli esempi di classe 0
num_class_0 = sum(1 for item in data if "Infiltration" not in item["labels"])
# Conta gli esempi di classe 1
num_class_1 = len(class_1_paths)
# Numero di esempi da generare per bilanciare
num_to_augment = num_class_0 - num_class_1
print(f"Esempi da augmentare: {num_to_augment}")

# Dataset augmentato per la classe 1
augmented_class_1_dataset = class_1_dataset.map(preprocess_and_augment, num_parallel_calls=tf.data.AUTOTUNE)
print(len(augmented_class_1_dataset))
augmented_class_1_dataset = augmented_class_1_dataset.repeat().take(num_to_augment)
# Ripeti per bilanciare
augmented_class_1_dataset_nobatch = augmented_class_1_dataset
print(len(augmented_class_1_dataset_nobatch), len(dataset_nobatch))
augmented_class_1_dataset = augmented_class_1_dataset.batch(64).prefetch(tf.data.AUTOTUNE)
print(len(augmented_class_1_dataset), len(dataset))

Esempi da augmentare: 3672
967
3672 5606
58 88


In [ ]:
def count_samples(dataset):
    count = 0
    for _ in dataset:
        count += 1
    return count

In [ ]:
# Combina dataset originale e augmentato
balanced_dataset = dataset_nobatch.concatenate(augmented_class_1_dataset_nobatch)
# Conta il numero totale di campioni
total_samples = count_samples(balanced_dataset)
print(f"Numero totale di campioni: {total_samples}, Numero totale atteso: {num_class_1 + num_class_0 + num_to_augment}")

Numero totale di campioni: 9278, Numero totale atteso: 9278


In [ ]:
class_counts = {"class_0": 0, "class_1": 0}

for _, label in balanced_dataset:
    if label.numpy() == 0:
        class_counts["class_0"] += 1
    else:
        class_counts["class_1"] += 1

print(f"Distribuzione delle classi nel dataset bilanciato: {class_counts}")

Distribuzione delle classi nel dataset bilanciato: {'class_0': 4639, 'class_1': 4639}


In [ ]:
balanced_dataset = balanced_dataset.shuffle(buffer_size=1000)
balanced_dataset = balanced_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

# TensorFlow Keras  
#### (with new dataset)

In [ ]:
dataset_size = len(balanced_dataset)
test_size = int(dataset_size * 0.2)
validation_size = int(dataset_size * 0.1)
train_size = dataset_size - test_size - validation_size

test_set = balanced_dataset.take(test_size)  # Primi 20% per il test
validation_set = balanced_dataset.skip(test_size).take(validation_size)  # Successivi 20% per validazione
train_set = balanced_dataset.skip(test_size + validation_size)  # Resto per il training

In [ ]:
print(f"Dataset totale: {dataset_size}")
print(f"Test set: {test_size}")
print(f"Validation set: {validation_size}")
print(f"Training set: {train_size}")

Dataset totale: 145
Test set: 29
Validation set: 14
Training set: 102


In [ ]:
history = model.fit(train_set, epochs=5, validation_data=validation_set)

Epoch 1/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 398s 4s/step - accuracy: 0.7982 - loss: 0.4501 - val_accuracy: 0.1585 - val_loss: 2.9227
Epoch 2/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 438s 4s/step - accuracy: 0.6894 - loss: 0.8967 - val_accuracy: 0.1540 - val_loss: 2.6816
Epoch 3/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 447s 4s/step - accuracy: 0.6891 - loss: 0.8501 - val_accuracy: 0.1518 - val_loss: 2.6926
Epoch 4/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 390s 4s/step - accuracy: 0.6837 - loss: 0.8549 - val_accuracy: 0.1618 - val_loss: 2.8316
Epoch 5/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 394s 4s/step - accuracy: 0.6817 - loss: 0.8980 - val_accuracy: 0.1607 - val_loss: 2.5189


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 105s 3s/step - accuracy: 0.1441 - loss: 2.5362
Test Loss: 2.5373177528381348, Test Accuracy: 0.14709052443504333


In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

print(classification_report(y_true, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
